In [1]:
import torch
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
    print("Memory Allocated:", torch.cuda.memory_allocated(0))

CUDA Available: True
GPU Name: NVIDIA GeForce RTX 5090
Memory Allocated: 0


In [2]:
import json
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch. utils.data import Dataset
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import MessagePassing, global_mean_pool
from pymatgen.core import Structure, Element

group_number = 18
row_number = 7
X_class_number = 10
r_class_number = 10
node_dim = group_number + row_number + X_class_number + r_class_number

distance_class_number = 10
edge_dim = distance_class_number

learned_dim = 20


# Step 1: Data Preparation

- Use pymatgen to load the json_gz file and pretty print the content
- Use Networkx to visualize the structure




## dataset

- Input matrix should contains information about nodes and edges.
- We only have element names, magnitude momentum of atoms, coordination of atoms to use:
- we should compute the distance of nearest atoms to fullfill the edges matrix
- for nodes information, use one-hot encoding to fullfill the nodes part


In [3]:
import numpy as np
path = './matbench_phonons.json/phonons.json'

class DataTransform(Dataset):
    def __init__(self, path, normalize = True):
        super().__init__()
        with open(path) as f:
            obj = json.load(f)
        self.index = obj["index"]
        self.data = obj["data"]
        max_index = max(self.index) + 1 if self.index else 0
        self.struct_list = [None] * max_index    
        for idx, data_item in zip(self.index, self.data):
            self.struct_list[idx] = Structure.from_dict(data_item[0])
        
        self.normalize = normalize
        if self.normalize:
            all_labels = [self.data[idx][1] for idx in self.index]
            self.label_mean = np.mean(all_labels)
            self.label_std = np.std(all_labels)
        else:
            self.label_mean = 0.0
            self.label_std = 1.0

    def __len__(self):  # 必须实现
        return len(self.index)
    
    def build_node(self, idx):
        struct = self.struct_list[idx]
        tensor_list = []
        for site in struct.sites:
            element = Element(site.specie.symbol)
            radius = element.atomic_radius * 100  # convert to pm
            # OneHot encoding with group and row:
            group = element.group
            row = element.row
            X = element.X
            # OneHot encoding with group and row:
            group = F.one_hot(torch.tensor(group - 1), num_classes=18).float()
            row = F.one_hot(torch.tensor(row - 1), num_classes=7).float()
            if(25<=radius<250):
                radius = F.one_hot(torch.tensor(int((radius - 25)/22.5)), num_classes=10).float()
            else:
                radius = torch.zeros(10)
            if (X < 0.5 or X >= 4.0):
                X =torch.zeros(10)
            else:
                X = F.one_hot(torch.tensor(int((X - 0.5) / 0.35)), num_classes=10).float()
            # concate group and row features
            node_feat = torch.cat([group, row, X, radius], dim=0)
            # print(node_feats): 
            # tensor([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,  # group 8
            #         0, 0, 1, 0, 0, 0, 0,        # row 3
            #         0, 0, 0, 0, 1, 0, 0, 0, 0, 0])          # X
            
            # concate group and row features 
            
            tensor_list.append(node_feat)
        node_feats = torch.stack(tensor_list, dim=0)
        return node_feats
    
    def build_edge(self, idx, cutoff=5.0):
        struct = self.struct_list[idx]
        num_atoms = len(struct.sites)
        nbrs = struct.get_all_neighbors(cutoff)
        tensor_list = []
        edge_index = torch.zeros((2,0), dtype=torch.long)
        for center_idx, neighbors in enumerate(nbrs):
            for neighbor in neighbors:
                distance = neighbor.nn_distance
                if (distance < 0.7 or distance >= 5.2):
                    distance = torch.zeros(10)
                else:
                    distance = F.one_hot(torch.tensor(int((distance-0.7)/0.45)), num_classes=10).float()
                tensor_list.append((distance))
                edge_index = torch.cat([edge_index, torch.tensor([[center_idx],[neighbor.index]])], dim=1)
            
        edge_fea = torch.stack(tensor_list, dim=0)
        return edge_index, edge_fea
    
    def __getitem__(self,idx):
        node_fea = self.build_node(idx)
        edge_index, edge_fea = self.build_edge(idx)
        label = self.data[idx][1]
        if self.normalize:
            label = (label - self.label_mean) / self.label_std
        y = torch.tensor([label], dtype=torch.float32)
        data = Data(
            x=node_fea,      
            edge_index=edge_index, 
            edge_attr=edge_fea,
            y=y
        )
        return data
    
'''    
    def build_visualized_graph(self, idx, cutoff=5.0):
        struct = self.struct_list[idx]
        num_atoms = len(struct.sites)
        G = nx.Graph()
        for i in range(num_atoms):
            G.add_node(i, element=struct.sites[i].specie.symbol)
        nbrs = struct.get_all_neighbors(cutoff)
        for center_idx, neighbors in enumerate(nbrs):
            for neighbor in neighbors:
                neighbor_idx = neighbor.index
                if not G.has_edge(center_idx, neighbor_idx):
                   G.add_edge(center_idx, neighbor_idx, distance=neighbor.nn_distance)
        return G
'''    

try:
    tryone = DataTransform(path)
    
    x, edge_index, edge_attr, y = tryone[0].x, tryone[0].edge_index, tryone[0].edge_attr, tryone[0].y
    print(x)
    print(edge_index)
    print(edge_attr)
    print(y)
except Exception as e:
    print(f"运行出错：{e}")


tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0.]])
tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]])
tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0.,

## Train_test split

In [4]:
from sklearn.model_selection import train_test_split

full_data = DataTransform(path)

train_size = int(0.8 * len(full_data))
test_size = len(full_data) - train_size

train_data, test_data = torch.utils.data.random_split(
    full_data, 
    [train_size, test_size],
    generator=torch.Generator().manual_seed(42)  # 固定随机种子
)

## Dataloader

In [5]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Step 2: Model construction

- Build the CGCNN model using multiple CGCNNLayer layers
- Number of layers:
- - GCNNNLayer: 2
- - Fully connected layers: 2
- - Pooling layer: global mean pooling

In [9]:
from torch_geometric.nn import GATConv, TransformerConv

class Model(nn.Module):
    def __init__(self, 
                 node_input_dim = node_dim,
                 edge_input_dim = edge_dim,
                 hidden_dim = 64,
                 num_conv_layers = 4,
                 dropout = 0.1
                 ):
        super(Model, self).__init__()

        self.node_embedding = nn.Sequential(
            nn.Linear(node_input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        self.edge_embedding = nn.Sequential(
            nn.Linear(edge_input_dim, hidden_dim),
            nn.ReLU()
        )
        self.conv_layers = nn.ModuleList([
            GATConv(
                hidden_dim, 
                hidden_dim, 
                heads=4,  # 多头注意力
                concat=False,
                edge_dim=hidden_dim,
                dropout=0.1
            )
            for _ in range(num_conv_layers)
        ])
        self.batch_norms = nn.ModuleList([
            nn.BatchNorm1d(hidden_dim) 
            for _ in range(num_conv_layers)
        ])

        self.fc = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_dim, hidden_dim // 2),
            nn.BatchNorm1d(hidden_dim // 2),
            nn.ReLU(),
            nn.Dropout(dropout),
            
            nn.Linear(hidden_dim // 2, 1)
        )

    def forward(self, data):
        x, edge_index, edge_attr = data.x, data.edge_index, data.edge_attr
        batch = data.batch  
        x = self.node_embedding(x)
        edge_attr = self.edge_embedding(edge_attr)
        for conv, bn in zip(self.conv_layers, self.batch_norms):
            identity = x
            x = conv(x, edge_index, edge_attr)
            x = bn(x)
            x = F.relu(x)
            x = x + identity 
        
        x = global_mean_pool(x, batch)
        
        # Output Layer
        x = self.fc(x)
        return x

# Step 3: Training

## Optimizer and Scheduler

In [10]:
model = Model()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"正在使用的设备: {device}")
model = model.to(device) 

criterion = nn.MSELoss()
criterion = criterion.to(device)  

import torch.optim as optim


optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=10)



正在使用的设备: cuda


## Early Stopping
AI 说的，早停，减少过拟合/梯度消失

In [11]:
class EarlyStopping:
    def __init__(self, patience=20, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        
    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

early_stopping = EarlyStopping(patience=20)

In [12]:
model.train()

epochs = 200
for epoch in range(epochs):
    total_loss = 0.0
    
    for data in train_loader:
        optimizer.zero_grad()
        
        data = data.to(device)
        label = data.y.to(device)
        
        # forward pass
        output = model(data)
        loss = criterion(output.squeeze(), label)

        # backward pass and optimization
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0) # AI 说这里加一个梯度裁剪
        optimizer.step()
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    scheduler.step(avg_loss)  # 根据当前损失调整学习率
    early_stopping(avg_loss)
    if early_stopping.early_stop:
        print(f"Early stopping at epoch {epoch+1}")
        break
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {avg_loss:.4f}')

Epoch [1/200], Loss: 0.7276
Epoch [2/200], Loss: 0.3417
Epoch [3/200], Loss: 0.2539
Epoch [4/200], Loss: 0.2672
Epoch [5/200], Loss: 0.2276
Epoch [6/200], Loss: 0.1865
Epoch [7/200], Loss: 0.1676
Epoch [8/200], Loss: 0.1872
Epoch [9/200], Loss: 0.1775
Epoch [10/200], Loss: 0.1632
Epoch [11/200], Loss: 0.1547
Epoch [12/200], Loss: 0.1483
Epoch [13/200], Loss: 0.1436
Epoch [14/200], Loss: 0.1363
Epoch [15/200], Loss: 0.1516
Epoch [16/200], Loss: 0.1538
Epoch [17/200], Loss: 0.1481
Epoch [18/200], Loss: 0.1264
Epoch [19/200], Loss: 0.1139
Epoch [20/200], Loss: 0.0953
Epoch [21/200], Loss: 0.1043
Epoch [22/200], Loss: 0.1364
Epoch [23/200], Loss: 0.1246
Epoch [24/200], Loss: 0.1402
Epoch [25/200], Loss: 0.1045
Epoch [26/200], Loss: 0.1131
Epoch [27/200], Loss: 0.0933
Epoch [28/200], Loss: 0.1266
Epoch [29/200], Loss: 0.1062
Epoch [30/200], Loss: 0.1104
Epoch [31/200], Loss: 0.1250
Epoch [32/200], Loss: 0.1115
Epoch [33/200], Loss: 0.0937
Epoch [34/200], Loss: 0.1124
Epoch [35/200], Loss: 0

# step 4: Testing

这个评估指标是agent写的

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def test(model, loader, dataset, device):
    model.eval()
    predictions = []
    targets = []
    
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data)
            
            # 反归一化
            if hasattr(dataset, 'dataset'):
                label_mean = dataset.dataset.label_mean
                label_std = dataset.dataset.label_std
            else:
                label_mean = dataset.label_mean
                label_std = dataset.label_std
            output = output * label_std + label_mean
            target = data.y * label_std + label_mean
            
            predictions.extend(output.cpu().numpy())
            targets.extend(target.cpu().numpy())
    
    predictions = np.array(predictions).flatten()
    targets = np.array(targets).flatten()
    
    # 计算评估指标
    mae = mean_absolute_error(targets, predictions)
    rmse = np.sqrt(mean_squared_error(targets, predictions))
    r2 = r2_score(targets, predictions)
    
    print(f"\n测试集评估:")
    print(f"  MAE:   {mae:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  R²:   {r2:.4f}")
    
    return predictions, targets

# 使用
predictions, targets = test(model, test_loader, test_data, device)


测试集评估:
  MAE:   53.2463
  RMSE: 92.6498
  R²:   0.9625


In [14]:
model.eval()
with torch.no_grad():
    for data in test_loader:
        data = data.to(device)
        output = model(data)
        output = output * full_data.label_std + full_data.label_mean
        print(output)

tensor([[ 642.8277],
        [ 525.6906],
        [ 309.9766],
        [ 221.5903],
        [ 794.6644],
        [1743.1216],
        [ 813.2318],
        [ 694.8705],
        [ 226.4801],
        [1288.0330],
        [1594.4539],
        [ 623.2313],
        [ 633.3937],
        [ 291.9004],
        [ 681.1582],
        [ 592.4925],
        [ 449.8996],
        [ 455.0058],
        [ 226.3767],
        [ 740.4271],
        [ 192.4677],
        [ 371.3585],
        [ 263.1004],
        [ 461.0519],
        [ 313.2459],
        [ 443.3501],
        [ 939.6577],
        [ 723.4598],
        [ 570.8029],
        [ 647.4584],
        [ 316.5052],
        [1195.9561]], device='cuda:0')
tensor([[ 235.4214],
        [ 210.3370],
        [ 430.1396],
        [ 219.9712],
        [ 350.2084],
        [ 195.5072],
        [ 246.6930],
        [ 821.3751],
        [ 492.9871],
        [ 578.8225],
        [ 630.8176],
        [ 331.5377],
        [ 146.9274],
        [ 243.4542],
        [ 294.19